In [6]:
import numpy as np
import pandas as pd
import os
import pickle
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
sys.path.append('/Users/ramtinyazdanian/PycharmProjects/job_skill_trend_analysis/')
from utilities.common_utils import *
from utilities.analysis_utils import *
from utilities.pandas_utils import *
from utilities.survey_response_utils import *
from utilities.constants import *
from utilities.params import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scipy.stats import shapiro, anderson, probplot
from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data

## Ad data

In [2]:
# Here, we will preprocess ad data, just like we had in the previous notebook.

## Survey data

## Auxiliary firm data

# Computing survey ground truth

The time period settings (and other settings) should be set through utilities/constants.py.

In [ ]:
# Use assemble_question_responses (or get_responses_in_rows) followed by find_majority_response_ground_truth 
# on the skills.

Let's see some of the ground truth emerging skills for 2017-2020.

### Looking at inter-rater agreement for the skills

We use Krippendorf's alpha.

In [1]:
# simply use get_interrater_agreement from survey_response_utils

### Creating a train/validation/test split for the evaluation of the pop methods and weights

Doing a train/validation/test split here makes sense because we are essentially training each model (rawpop, logpop, binpop) on the training data (i.e. survey ground truth) by choosing the weights (through training) and thresholds (hyperparameters set using the validation set or CV) based on F1 on the training data. However, this is a strong assumption about the survey data: that they actually _are_ ground truth, and that training a model on part of them and testing it on another does conceptually make sense. This assumption is only valid if we have enough survey data to create a respectable train/validation/test split.

On the other hand, this is our only choice, because choosing the weights based on _all_ the survey data, and then presenting the F1 as our metric makes no sense, as it's basically just reporting the training error as the "error", which would have nothing to do with the model's ability to generalise. So we have the validation set (or CV) to find the best hyperparameters, then we train the weights on all the training data and use the test set to see the performance of the model on _data it has never seen in training_. This will let us see how well we can, in the best case, approximate the results of the survey. Just the usual stuff.

Now, the important question is how we create the split: do we split by person (i.e. each set is made up of the full responses of several people), by skill (i.e. each set is made up of the final ground truth for a set of skills), or by individual skill responses (i.e. some of the responses for a skill could be in one set and some in another). Among these three, only the second makes sense, as the first and third can lead to serious inconsistencies (e.g. a skill could be Yes in one set and No in the other). So **we split by skill**.

Another thing to remember is that the precision and recall only matter on the positive class and not on the negative class, as the negative class is sort of the "default" and much bigger than the positive class.

Finally, a critical question is our treatment of skills that _weren't in the survey_. Do we consider them to be negatives? If we do, we may be assuming too much, since some of them might have been emerging skills if we'd asked people about them. On the other hand, if we exclude them, then our set is limited to skills that were in the survey, which is a small set. However, a nice solution to this issue is to only count the TP and FN rate (true positive and false negative). This requires literally nothing but "which Yes skills did we get" and "which Yes skills did we miss". However, it disregards the number of skills that each method produces as its results, which is an important thing when comparing different thresholds (since more restrictive thresholds will naturally produce fewer "emerging skills" and they would obviously perform worse, even though their recall or precision may have been better). An alternative solution is to compute and present _all of these things_, meaning:
* TP and FN rates
* Precision, Recall, and F1 assuming Full-Unsures are "No"s and Out-of-survey skills don't exist.
* Precision, Recall, and F1 assuming Full-Unsure and Out-of-survey skills are "No"s.

Out of these three, the first two are more valid, since the assumption that all Out-of-surveys are "No"s may be too big. Since we're going to be training an _actual classifier model_ on the data, I think we're going to have to choose, and to me, what makes the most sense is to go with the second option, as using the third option will beg the question "what did you do all this survey stuff for then".

# Investigating the popularity profiles of several actual emerging skills

In [4]:
# Call investigate_skill_pop_profile(df, skill, pop_type, time_period, normaliser, smooth) 
# on several skills and plot the results.

# The decision regarding the types of features to be used in the computation of emerging skills is made here.

# Computing emerging skills and evaluating them

We have a train/validation/test split (or a train/test split if we want to do cross-validation), and now it's time to define what exactly our training process is like, and what the feature weights, objective, and hyperparameters are.

* Data points: Data points are as follows: Each skill-period pair constitutes one data point. What does this mean? It means that tensorflow_2017-2018 is one data point, and tensorflow_2017-2020 is another, et cetera. This means that we do not have to treat each time period as a separate entity and only deal with one dataset. If we didn't do this, we would have to train a separate model for each period, giving us 6 models with 6 weight vectors, and then we would have to average their F1-scores. The advantage of having one model is that we have more training data for the model, but the disadvantage is how much the "i.i.d." assumption is violated when we have 2017-2018 data _and_ 2017-2019 data _and_ 2017-2020 data etc. all as part of the input for _one_ model. Still, training data is very limited and I consider this combined model preferable.

* Features and output: The features are extracted for every single skill-period based on its pop profile (depending on the method chosen), yielding a n_skills * n_features matrix. The output variable for each data point is 1 if the skill has been considered emerging (by our respondents) in that time period, 0 if not. Full-Unsure skills should be considered "No"s in order not to limit the training set too much, but we may have to take them out of the set.

* Weights: The weights are the vector whose dot product with each skill-period's feature vector yields its **score**. The score is then subjected to the score lower bound, and the skill's rawpop is also checked against the rawpop upper bound.

* Other model parameters (not hyperparameters): The score lower bound can be considered to be another model parameter, which we would train alongside the weights (e.g. it could be integrated into the feature weights by adding a constant 1 feature, or it could be alternately optimised). The lower bound would not necessarily be a quantile, since we're optimising it anyway and the constraint that it be a specific quantile is unnecessary. The alternative is to treat the score lower bound as a hyperparameter, which I think is a bad idea.

* Hyperparameters: The score lower bound can be a hyperparameter. The rawpop upper bound is definitely a hyperparameter, since the quantity it thresholds is, unlike the score, predetermined and thus constant for each model and making it a hyperparameter immensely simplifies things (since it just makes a bunch of skills instant negatives).

* Train/test split: We have a training/test split on our data (6 splits if we have one model per period, and this is for each pop method). The training data can further be split into train/validation (or we could do cross-validation) to first find the best hyperparameters (per model) and then we would train the model on the entire training set using the best hyperparameters and look at its performance on the test set to see how well it manages to generalise to the hidden results of the survey. Based on the models' performance on the test set, we choose the _best model_ that we will use for the rest of the study (and at this point the model is already trained, I don't think we'd want to now retrain that model with its best hyperparameters on the entire dataset, since this kinda defeats the purpose of a train/test split).

#### Pros and cons of a single classifier vs one per period

Pros of a single classifier:
* Can generalise to other time periods as it is not tied to one specific period.
* The use of a score lower bound _value_ (as opposed to quantile) makes sense since the score itself isn't a directly interpretable quantity, unlike velocity/acceleration/other raw features. We could then investigate the value of that bound just like the weights of the other features.
* More data means a better classifier and the ability to have more features without an elevated risk of overfitting.
* Allows us to see which features are _globally_ more important, instead of giving us specific information about each period.

Cons of a single classifier:
* Different time periods may have different features, which could lead to score gaps and thus, unbalanced error rates for data coming from different time periods.

Pros of per-period classifiers:
* No score gap issue, as every period is treated separately.
* Can use quantile score gap (although this isn't necessarily even important).

Cons of per-period classifiers:
* Generalisation to other periods is not straightforward (or in other words, which classifier are you gonna choose?).
* Small dataset for each period, high risk of overfitting.
* Important features become specific to time period.

Looking at all these pros and cons, I think that the preferred method should be a single classifier.

## Global preprocessing

In [ ]:
# Run reformat_y on the y ground truth dataframe.

## Rawpop

### Preprocessing

In [ ]:
# Run compute_total_poptype_mean with pop_type='raw' (for the normalisation) on original ad df
# Run compile_all_feature_dfs with pop_type='raw' on skills ad df (provide list of extraction methods 
# to feature_types)

# Run skills_to_indices to generate indices.
# Run create_train_test_split to create the training and test sets.
# Run generate_cv_folds on the training data to get... cv folds.

### Computation (training and validation)

In [ ]:
# Run cross_validate_with_quantile with the appropriate feature column name (e.g. 'linreg', 'tsfresh'). 
# Set C_LIST and QUANTILES through utilities/params.py

### Evaluation (test and manual investigation)

In [ ]:
# Run predict_and_evaluate_dfs using the best model from above and on the test set.

## Logpop

### Preprocessing

In [ ]:
# Run compute_total_poptype_mean with pop_type='log' (for the normalisation) on original ad df
# Run compile_all_feature_dfs with pop_type='log' on skills ad df

# Run skills_to_indices to generate indices.
# Run create_train_test_split to create the training and test sets.
# Run generate_cv_folds on the training data to get... cv folds.

### Computation (training and validation)

In [ ]:
# Run cross_validate_with_quantile with the appropriate feature column name. 
# Set C_LIST and QUANTILES through utilities/params.py

### Evaluation (test and manual investigation)

## Binpop

### Preprocessing

In [ ]:
# Run compute_total_poptype_mean with pop_type='bin' (for the normalisation) on original ad df
# Run compile_all_feature_dfs with pop_type='bin' on skills ad df

# Run skills_to_indices to generate indices.
# Run create_train_test_split to create the training and test sets.
# Run generate_cv_folds on the training data to get... cv folds.

### Computation (training and validation)

In [ ]:
# Run cross_validate_with_quantile with the appropriate feature column name. 
# Set C_LIST and QUANTILES through utilities/params.py

### Evaluation (test and manual investigation)

## The winning method

We now have a winning method + hyperparameters + weights from the previous step, and the best model is trained on the entire training set. Now, we use this model to compute the emerging skills for every period.

### Looking at feature importance

# Creating sets of firms

## HITS on emerging skill sets

## POP based on ad counts

## REV based on revenue (different groupings)

## EMC based on employee count (different groupings)